In [0]:
# SILVER LAYER - Data Cleaning, Validation, and Enrichment
# Purpose: Create clean, joined, business-ready tables

from pyspark.sql import functions as F
from pyspark.sql.window import Window

print("Starting Silver Layer transformations...")

# ============================================
# 1. CLEAN ORDERS TABLE
# ============================================

print("\n1. Processing Orders...")

orders = spark.table("bronze_orders")

# Check initial count
print(f"   Raw orders: {orders.count()}")

# Filter only delivered orders
orders_clean = orders.filter(F.col("order_status") == "delivered")
print(f"   Delivered orders: {orders_clean.count()}")

# Convert timestamps
date_columns = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

for col in date_columns:
    orders_clean = orders_clean.withColumn(col, F.to_timestamp(col))

# Calculate delivery time in days
orders_clean = orders_clean.withColumn(
    "delivery_days",
    F.datediff(F.col("order_delivered_customer_date"), F.col("order_purchase_timestamp"))
)

# Calculate delivery delay (negative = early, positive = late)
orders_clean = orders_clean.withColumn(
    "delivery_delay_days",
    F.datediff(F.col("order_delivered_customer_date"), F.col("order_estimated_delivery_date"))
)

orders_clean.write.format("delta").mode("overwrite").saveAsTable("silver_orders")
print("   ✓ Created silver_orders")

# ============================================
# 2. ENRICH ORDER ITEMS WITH PRODUCT INFO
# ============================================

print("\n2. Processing Order Items with Products...")

order_items = spark.table("workspace.default.bronze_order_items")
products = spark.table("workspace.default.bronze_products")
translations = spark.table("workspace.default.bronze_product_category_translation")

print(f"   Order items: {order_items.count()}")

# Join products with translations (remove ingestion_timestamp to avoid conflicts)
products_enriched = products.join(
    translations,
    products.product_category_name == translations.product_category_name,
    "left"
).select(
    products.product_id,
    products.product_category_name,
    products.product_name_lenght,
    products.product_description_lenght,
    products.product_photos_qty,
    products.product_weight_g,
    products.product_length_cm,
    products.product_height_cm,
    products.product_width_cm,
    translations.product_category_name_english.alias("category_english")
)

# Join order items with enriched products (drop ingestion_timestamp from order_items to avoid duplicates)
order_items_clean = order_items.drop("ingestion_timestamp")

order_items_enriched = order_items_clean.join(
    products_enriched,
    "product_id",
    "left"
)

order_items_enriched.write.format("delta").mode("overwrite").saveAsTable("workspace.default.silver_order_items")
print("   ✓ Created silver_order_items")

# ============================================
# 3. AGGREGATE PAYMENTS BY ORDER
# ============================================

print("\n3. Processing Payments...")

payments = spark.table("bronze_order_payments")
print(f"   Payment records: {payments.count()}")

# Aggregate payments per order
payments_agg = payments.groupBy("order_id").agg(
    F.sum("payment_value").alias("total_payment_value"),
    F.count("*").alias("payment_installments_count"),
    F.first("payment_type").alias("primary_payment_type")
)

payments_agg.write.format("delta").mode("overwrite").saveAsTable("silver_order_payments")
print("   ✓ Created silver_order_payments")

# ============================================
# 4. AGGREGATE REVIEWS BY ORDER
# ============================================

print("\n4. Processing Reviews...")

reviews = spark.table("bronze_order_reviews")
print(f"   Review records: {reviews.count()}")

# Use try_cast for handling malformed values
reviews = reviews.withColumn(
    "review_score_clean", 
    F.expr("try_cast(review_score as double)")
)

# Filter valid score-only reviews
reviews_valid = reviews.filter(F.col("review_score_clean").isNotNull())

# Clean and aggregate reviews
reviews_agg = reviews_valid.groupBy("order_id").agg(
    F.avg("review_score_clean").alias("avg_review_score"),
    F.max("review_score_clean").alias("max_review_score"),
    F.min("review_score_clean").alias("min_review_score"),
    F.count("*").alias("review_count")
)

reviews_agg.write.format("delta").mode("overwrite").saveAsTable("silver_order_reviews")

# Quality check: show how many records were filtered
total_reviews = reviews.count()
valid_reviews = reviews_valid.count()
print(f"   Total reviews: {total_reviews}")
print(f"   Valid reviews: {valid_reviews}")
print(f"   Invalid reviews (filtered out): {total_reviews - valid_reviews}")

print("   ✓ Created silver_order_reviews")

# ============================================
# 5. CREATE MASTER ORDER TABLE (FACT TABLE)
# ============================================

print("\n5. Creating Master Orders Fact Table...")

# Start with clean orders
master = spark.table("silver_orders")

# Drop ingestion_timestamp from customers before join
customers = spark.table("bronze_customers").drop("ingestion_timestamp")
master = master.join(customers, "customer_id", "left")

# Drop ingestion_timestamp from payments_agg before join
payments_agg = spark.table("silver_order_payments").drop("ingestion_timestamp")
master = master.join(payments_agg, "order_id", "left")

# Drop ingestion_timestamp from reviews_agg before join
reviews_agg = spark.table("silver_order_reviews").drop("ingestion_timestamp")
master = master.join(reviews_agg, "order_id", "left")

# Drop ingestion_timestamp from silver_order_items before aggregation
order_metrics = spark.table("silver_order_items").drop("ingestion_timestamp").groupBy("order_id").agg(
    F.sum("price").alias("total_items_price"),
    F.sum("freight_value").alias("total_freight_value"),
    F.count("*").alias("items_count"),
    F.countDistinct("product_id").alias("unique_products_count")
)
master = master.join(order_metrics, "order_id", "left")

# Calculate total order value
master = master.withColumn(
    "order_total_value",
    F.coalesce(F.col("total_payment_value"), F.col("total_items_price") + F.col("total_freight_value"))
)

master.write.format("delta").mode("overwrite").saveAsTable("silver_orders_master")
print(f"   ✓ Created silver_orders_master with {master.count()} orders")

# Show sample of master table
print("\nSample of silver_orders_master:")
spark.table("silver_orders_master").select(
    "order_id", "customer_unique_id", "order_total_value", 
    "delivery_days", "avg_review_score", "items_count"
).show(5)

print("\n🎉 Silver layer complete! Data is cleaned, joined, and ready for analytics.")

In [0]:
%sql
DROP TABLE IF EXISTS silver_orders;
DROP TABLE IF EXISTS workspace.default.silver_order_items;
DROP TABLE IF EXISTS silver_order_payments;
DROP TABLE IF EXISTS silver_order_reviews;
DROP TABLE IF EXISTS silver_orders_master;